In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[]


In [2]:
import keras
from keras.models import Sequential

Using TensorFlow backend.


In [3]:
vocablen = 10000
maxlen = 128 # for batched input
batch_size = 512

In [4]:
# vectorized data
# https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa
# save np.load
np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=vocablen, 
                                    seed=113,
                                    start_char=1,
                                    oov_char=2,
                                    index_from=3,
                                    )

np.load = np_load_old

17465344/17464789 [==============================] - 1s 0us/step


In [5]:
print(x_train[0], y_train[0])
print(x_test[0], y_test[0])
print(len(x_train), len(x_test))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] 1
[1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5, 4, 360, 7, 

In [6]:
# index to decode vectors into tokens and vice versa
# 0 is reserved
# word -> num
word_index = keras.datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 0s 0us/step


In [7]:
print(word_index['the'])
reversed_word_index = {v:k for k, v in word_index.items()}
print(reversed_word_index[1], reversed_word_index[2], reversed_word_index[3],)

1
the and a


In [8]:
# reserve special characters
word_index = {k:v+2 for k, v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<OOV>'] = 2

# limit number of words
word_index = {k:v for k, v in word_index.items() if v < vocablen}

In [9]:
print(word_index['<OOV>'], word_index['the'], len(word_index))

2 3 10000


In [10]:
# num -> word
reversed_word_index = {v:k for k, v in word_index.items()}

In [11]:
print(reversed_word_index[0], reversed_word_index[1], reversed_word_index[2], reversed_word_index[3])

<PAD> <START> <OOV> the


In [12]:
# pad samples
x_train = keras.preprocessing.sequence.pad_sequences(x_train, 
                            maxlen=maxlen,
                            padding='post',
                            truncating='post',
                            value=word_index['<PAD>']
                            )

x_test = keras.preprocessing.sequence.pad_sequences(x_test, 
                            maxlen=maxlen,
                            padding='post',
                            truncating='post',
                            value=word_index['<PAD>']
)

In [13]:
def decode(X):
    if not isinstance(X[0], list):
        # add batch dim
        X = [X]
    tokens = []
    for x in X:
        tokenized = [reversed_word_index.get(num-1, '<OOV>') for num in x]
        tokens.append(tokenized)
    return tokens

In [14]:
s = ' '.join(decode(x_train[0])[0])
print(s)

<PAD> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <START> is an amazing actor and now the same being director <START> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <START> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and


In [15]:
from keras.models import Sequential
from keras.layers import (
    Embedding,
    Bidirectional,
    Dense,
    LSTM,
    Masking,
)

model = Sequential([
    # mask before embedding
    Embedding(vocablen, 8),
    Masking(mask_value=word_index['<PAD>']),
    Masking(mask_value=word_index['<START>']),    
    Masking(mask_value=word_index['<OOV>']),
    Bidirectional(LSTM(64,
                return_sequences=True,
                )),
    Bidirectional(LSTM(32, 
                return_sequences=True,
                )),
    Bidirectional(LSTM(16)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid'),
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 8)           80000     
_________________________________________________________________
masking_1 (Masking)          (None, None, 8)           0         
_________________________________________________________________
masking_2 (Masking)          (None, None, 8)           0         
_________________________________________________________________
masking_3 (Masking)          (None, None, 8)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         37376     
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 64)          41216     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 32)                10368     
__________

In [16]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
epochs = 5
history = model.fit(x_train, y_train, 
      epochs=epochs, batch_size=batch_size, verbose=2,
      validation_split=0.5)

Train on 12500 samples, validate on 12500 samples
Epoch 1/5
 - 64s - loss: 0.6932 - acc: 0.5007 - val_loss: 0.6930 - val_acc: 0.5150
Epoch 2/5
 - 57s - loss: 0.6902 - acc: 0.5065 - val_loss: 0.6681 - val_acc: 0.5187
Epoch 3/5
 - 56s - loss: 0.5734 - acc: 0.6984 - val_loss: 0.4894 - val_acc: 0.7822
Epoch 4/5
 - 56s - loss: 0.4018 - acc: 0.8336 - val_loss: 0.4248 - val_acc: 0.8111
Epoch 5/5
 - 57s - loss: 0.2908 - acc: 0.8918 - val_loss: 0.4047 - val_acc: 0.8327


In [18]:
results = model.evaluate(x_test, y_test)
print(results)

25000/25000 [==============================] - 332s 13ms/step
[0.43191788868904113, 0.81504]


In [19]:
model.save('keras-sentiment-classifier.h5')